In [1]:
from frcnn.config import singleton_config
import keras.backend as K
import tensorflow as tf
import numpy as np

from frcnn.classifier import ClassifierNetwork
from frcnn.classifier_trainer import ClassifierTrainer
from frcnn.config import singleton_config, Config
from frcnn.fen import FeatureExtractionNetwork
from frcnn.rpn import RegionProposalNetwork
from frcnn.rpn_trainer import RPNTrainer
from frcnn.voc import PascalVocData

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Configuration

In [11]:
np.set_printoptions(edgeitems=10, linewidth=280)
np.core.arrayprint._line_width = 280

In [3]:
config = singleton_config()

# Momory Limit & Debugging
tf_config = tf.ConfigProto()
tf_config.gpu_options.per_process_gpu_memory_fraction = 0.8
tf_config.gpu_options.allow_growth = True

sess = tf.Session(config=tf_config)
# sess = tf_debug.LocalCLIDebugWrapperSession(sess)
K.set_session(sess)

# Initialization

In [4]:
# Load data
vocdata = PascalVocData(config.data_path)
train, test, class_mapping = vocdata.load_data(limit_size=30, add_bg=True)

# Load training tools
rpn_trainer = RPNTrainer(train, shuffle=True, augment=True)
clf_trainer = ClassifierTrainer(config, class_mapping)

# Create Model
fen = FeatureExtractionNetwork(config, input_shape=(None, None, 3))
rpn = RegionProposalNetwork(fen, config)

# Training

In [5]:
batch_img, batch_cls, batch_regr, img_meta = rpn_trainer.next_batch()
rpn_loss = rpn.model.train_on_batch(batch_img, [batch_cls, batch_regr])

In [7]:
cls_y = rpn.rpn_tensors['reg_cls_y']
cls_y2 = rpn.rpn_tensors['reg_cls_y2']



reg_cls_y, reg_cls_y2 = sess.run([cls_y, cls_y2], 
                                 feed_dict={rpn.fen.input_img: batch_img, 
                                            rpn.rpn_tensors['reg_y_true']: batch_regr})


In [24]:
[reg_cls_y != reg_cls_y2]


[array([[[[False, False, False, False, False, False, False, False, False, False, ..., False, False, False, False, False, False, False, False, False, False],
          [False, False, False, False, False, False, False, False, False, False, ..., False, False, False, False, False, False, False, False, False, False],
          [False, False, False, False, False, False, False, False, False, False, ..., False, False, False, False, False, False, False, False, False, False],
          [False, False, False, False, False, False, False, False, False, False, ..., False, False, False, False, False, False, False, False, False, False],
          [False, False, False, False, False, False, False, False, False, False, ..., False, False, False, False, False, False, False, False, False, False],
          [False, False, False, False, False, False, False, False, False, False, ..., False, False, False, False, False, False, False, False, False, False],
          [False, False, False, False, False, False, False